In [1]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
housing = pd.read_csv('../data/Ames_Housing_Price_Data.csv', index_col = 0)
real_estate = pd.read_csv('../data/Ames_Real_Estate_Data.csv')
housing_df = pd.merge(housing,real_estate,left_on='PID',right_on='MapRefNo',how='inner')

house_df = pd.read_csv('../data/final_df.csv', index_col = 0)

D:\software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Merging two dataset has 21 observations in difference (left/inner), so using 2558 observations to analyze housing price

In [14]:
# Checking house price dataset, contains 2580 observations
housing.shape

(2580, 81)

In [6]:
# Checking final_df after generating geoCode, found duplicated observations
house_df.shape

(2603, 179)

In [38]:
# Test cell, ignored! 
house_df_test = house_df.copy()
len(house_df_test.drop_duplicates(subset=['PID'],keep = 'first'))

2558

In [43]:
# drop duplicated observations
house_df = house_df.drop_duplicates(subset=['PID'],keep = 'first')
house_df.shape

(2558, 179)

In [53]:
# Train-test split 
train1 = house_df[house_df['YrSold']!=2010]
train2 = house_df[house_df['YrSold']==2010]
print(train1.shape)
print(train2.shape)
X_train = train1.drop(['SalePrice'],axis=1)
y_train = train1['SalePrice']
X_test = train2.drop(['SalePrice'],axis=1)
y_test = train2['SalePrice']

(2244, 179)
(314, 179)


In [58]:
X_train

,index,PID,GrLivArea,Distance,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,MasVnrArea2,GarageArea2,PoolArea2,HalfBath2,BsmtHalfBath2,BasmtFinSF1,BasmtFinSF2,BasmtFinSF,total_LivArea,num_bathroom,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,ClassSc_S,Legal_Pr,SchD_S,TxD_S,MA_Ownr1,MA_Ownr2,MA_Line1,MA_Line2,MA_City,MA_State,MA_Zip1,MA_Zip2,Rcrd_Yr,Rcrd_Mo,Inst1_No,Inst1_Yr,Inst1_Mo,Inst1TPr,LndAc_S,ImpAc_S,OthAc_S,TtlVal_AsrYr,ValType,X1TPr_D,X1TSc_D,X2TPr_D,X2TSc_D,X1TPr_S,X1TSc_S,X2TPr_S,X2TSc_S,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,ParType,BldgNo_S,DwlgNo_S,YrBuilt,Ext1,Ext2,Central Air,GLA,TtlBsmtSF,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,SaleCond,ParclRel,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,address,address2,address3,location2,point2,latitude2,longitude2,altitude2,ISU_lat_long,ISU_lat,ISU_long
1,1,905476230,1049,1.446325,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,1,266,0,0.0,0.0,552,393,945,1994,3.0,905476230,0,0,3416 WEST ST,RL,NaN,NaN,NaN,RESIDENTIAL,1-STORY TWNHM - 1946 & NEWER,WEST AMES ADD BEG NE COR LOT 12 BLK 1 S99.94' ...,1,1,"HUBERT, JOHN F & GRETTA R",NaN,3416 WEST ST,NaN,AMES,IA,50014.0,3541.0,2020,1,2020-00358,2020,1,WRDJSV,43500,0,130600,174100,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0,Dwg&Lot,0,1,1984.0,HdBoard,HdBoard,Yes,1049.0,1049.0,1984.0,1.0,2020.0,1.0,Normal,Deed,3416,NaN,WEST,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"3416 WEST ST, AMES, IA","3416 WEST ST, AMES, IA",3416,"3416, West Street, Ontario, Ames, Story County...","(42.024855315930424, -93.66367099341755, 0.0)",42.024855,-93.663671,0.0,"(42.02704410769817, -93.64641075013807)",42.027044,-93.646411
3,3,911128020,1001,2.740599,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,0,216,0,0.0,0.0,737,0,737,1738,1.0,911128020,0,0,320 S 2ND ST,S-SMD,NaN,NaN,NaN,RESIDENTIAL,1-STORY 1945 & OLDER,BLACKS 3RD ADD E60' W125' LOT 2 & LOT 3 BLK 7,1,1,"SKLENAR, TIM","CARNEY, SARAH",320 S 2ND ST,NaN,AMES,IA,50010.0,6702.0,2016,4,201603010,2016,4,WRDJSV,41100,0,123200,164300,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0,Dwg&Lot,0,1,1930.0,MetalSd,MetalSd,Yes,1001.0,837.0,1930.0,1.0,NaN,NaN,NaN,Deed,320,S,2ND,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"320 S 2ND ST, AMES, IA","320 S 2ND ST, AMES, IA",320,"320, South 2nd Street, Meadow Lane Mobile Home...","(42.021547615297024, -93.61406809992513, 0.0)",42.021548,-93.614068,0.0,"(42.02704410769817, -93.64641075013807)",42.027044,-93.646411
4,4,535377150,1039,3.050284,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Ol

In [55]:
house_df[house_df['PID'].duplicated()==True]

,index,PID,GrLivArea,SalePrice,Distance,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,MasVnrArea2,GarageArea2,PoolArea2,HalfBath2,BsmtHalfBath2,BasmtFinSF1,BasmtFinSF2,BasmtFinSF,total_LivArea,num_bathroom,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,ClassSc_S,Legal_Pr,SchD_S,TxD_S,MA_Ownr1,MA_Ownr2,MA_Line1,MA_Line2,MA_City,MA_State,MA_Zip1,MA_Zip2,Rcrd_Yr,Rcrd_Mo,Inst1_No,Inst1_Yr,Inst1_Mo,Inst1TPr,LndAc_S,ImpAc_S,OthAc_S,TtlVal_AsrYr,ValType,X1TPr_D,X1TSc_D,X2TPr_D,X2TSc_D,X1TPr_S,X1TSc_S,X2TPr_S,X2TSc_S,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,ParType,BldgNo_S,DwlgNo_S,YrBuilt,Ext1,Ext2,Central Air,GLA,TtlBsmtSF,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,SaleCond,ParclRel,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,address,address2,address3,location2,point2,latitude2,longitude2,altitude2,ISU_lat_long,ISU_lat,ISU_long


In [12]:
len(set(house_df['PID']))

2558

In [46]:
len(house_df[house_df['YrSold']==2010])

314

In [29]:
house_df[house_df['PID']==]

0.12403100775193798

In [63]:
X_train['BldgAge']

1        25
3        77
4       109
5         8
6         6
       ... 
2598     93
2599     54
2600     58
2601      7
2602     13
Name: BldgAge, Length: 2244, dtype: int64

## Yi's Feature engineer

In [61]:
# calculate age of building
X_train['BldgAge'] = X_train['YrSold'] - X_train['YearBuilt']

# convert MSSubClass to str
X_train['MSSubClass'] = X_train[['MSSubClass']].astype('str')

# convert NA's to 0s in LotFrontage
X_train['LotFrontage'] = np.where(np.isnan(X_train['LotFrontage']), 0, X_train['LotFrontage'])

# binarize YearRemodAdd
X_train['Remodeled'] = np.where(X_train['YearRemodAdd'] == X_train['YearBuilt'], 0, 1)

# binarize Alley
X_train['Alley'] = np.where(pd.isnull(X_train['Alley']), 0, 1)

# binarize MSSubClass to PUD or not PUD
X_train['IsPUD'] = np.where(X_train['MSSubClass'].isin(['120','150','160','180']), 1, 0)

# binarize LotShape to Reg or not Reg
X_train['LotIsReg'] = np.where(X_train['LotShape']=='Reg', 1, 0)

# binarize Condition1/2 to positive feature or no positive feature
X_train['PosFeat'] = np.where(X_train['Condition1'].isin(['PosN','PosA'])|\
                                X_train['Condition2'].isin(['PosN','PosA']), 1, 0)

# binarize condition 1/2 to normal sorroundings or not normal surroundings
X_train['SurrIsNormal'] = np.where((X_train['Condition1']=='Norm') & (X_train['Condition2']=='Norm'), 1, 0)

### These are the features I selected from the first 27 features: 
- GrLivArea
- LotArea
- BldgAge
- Remodeled
- IsPUD
- LotIsReg
- PosFeat
- SurrIsNormal (unsure about this one)

In [62]:
X_train

,index,PID,GrLivArea,Distance,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,MasVnrArea2,GarageArea2,PoolArea2,HalfBath2,BsmtHalfBath2,BasmtFinSF1,BasmtFinSF2,BasmtFinSF,total_LivArea,num_bathroom,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,ClassSc_S,Legal_Pr,SchD_S,TxD_S,MA_Ownr1,MA_Ownr2,MA_Line1,MA_Line2,MA_City,MA_State,MA_Zip1,MA_Zip2,Rcrd_Yr,Rcrd_Mo,Inst1_No,Inst1_Yr,Inst1_Mo,Inst1TPr,LndAc_S,ImpAc_S,OthAc_S,TtlVal_AsrYr,ValType,X1TPr_D,X1TSc_D,X2TPr_D,X2TSc_D,X1TPr_S,X1TSc_S,X2TPr_S,X2TSc_S,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,ParType,BldgNo_S,DwlgNo_S,YrBuilt,Ext1,Ext2,Central Air,GLA,TtlBsmtSF,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,SaleCond,ParclRel,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,address,address2,address3,location2,point2,latitude2,longitude2,altitude2,ISU_lat_long,ISU_lat,ISU_long,BldgAge,Remodeled,IsPUD,LotIsReg,PosFeat,SurrIsNormal
1,1,905476230,1049,1.446325,120,RL,42.0,4235,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,1,266,0,0.0,0.0,552,393,945,1994,3.0,905476230,0,0,3416 WEST ST,RL,NaN,NaN,NaN,RESIDENTIAL,1-STORY TWNHM - 1946 & NEWER,WEST AMES ADD BEG NE COR LOT 12 BLK 1 S99.94' ...,1,1,"HUBERT, JOHN F & GRETTA R",NaN,3416 WEST ST,NaN,AMES,IA,50014.0,3541.0,2020,1,2020-00358,2020,1,WRDJSV,43500,0,130600,174100,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0,Dwg&Lot,0,1,1984.0,HdBoard,HdBoard,Yes,1049.0,1049.0,1984.0,1.0,2020.0,1.0,Normal,Deed,3416,NaN,WEST,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"3416 WEST ST, AMES, IA","3416 WEST ST, AMES, IA",3416,"3416, West Street, Ontario, Ames, Story County...","(42.024855315930424, -93.66367099341755, 0.0)",42.024855,-93.663671,0.0,"(42.02704410769817, -93.64641075013807)",42.027044,-93.646411,25,0,1,1,0,1
3,3,911128020,1001,2.740599,30,C (all),60.0,6060,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,0,216,0,0.0,0.0,737,0,737,1738,1.0,911128020,0,0,320 S 2ND ST,S-SMD,NaN,NaN,NaN,RESIDENTIAL,1-STORY 1945 & OLDER,BLACKS 3RD ADD E60' W125' LOT 2 & LOT 3 BLK 7,1,1,"SKLENAR, TIM","CARNEY, SARAH",320 S 2ND ST,NaN,AMES,IA,50010.0,6702.0,2016,4,201603010,2016,4,WRDJSV,41100,0,123200,164300,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0,Dwg&Lot,0,1,1930.0,MetalSd,MetalSd,Yes,1001.0,837.0,1930.0,1.0,NaN,NaN,NaN,Deed,320,S,2ND,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"320 S 2ND ST, AMES, IA","320 S 2ND ST, AMES, IA",320,"320, South 2nd Street, Meadow Lane Mobile Home...","(42.021547615297024, -93.61406809992513, 0.0)",42.021548,-93.614068,0.0,"(42.02704410769817, -93.64641075013807)",42.027044,-93.646411,77,1,0,1,0,1
4,4,5

In [57]:
house_df['YrSold']

0       2010
1       2009
3       2007
4       2009
5       2009
        ... 
2598    2009
2599    2009
2600    2007
2601    2007
2602    2006
Name: YrSold, Length: 2558, dtype: int64

### 2580 to 2407 observations by getting rid of the house not in Ames city.

In [5]:
house_df.corr()

,index,PID,GrLivArea,SalePrice,Distance,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MasVnrArea2,GarageArea2,PoolArea2,HalfBath2,BsmtHalfBath2,BasmtFinSF1,BasmtFinSF2,BasmtFinSF,total_LivArea,num_bathroom,GeoRefNo,Tier,Range,SchD_S,TxD_S,MA_Line2,MA_Zip1,MA_Zip2,Rcrd_Yr,Rcrd_Mo,Inst1_Yr,Inst1_Mo,LndAc_S,ImpAc_S,OthAc_S,TtlVal_AsrYr,X2TPr_D,X2TSc_D,X2TPr_S,X2TSc_S,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,BldgNo_S,DwlgNo_S,YrBuilt,GLA,TtlBsmtSF,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,PA-Nmbr,PA-PostD,NmbrBRs,address3,latitude2,longitude2,altitude2,ISU_lat,ISU_long
index,1.000000e+00,2.473041e-02,-1.517788e-02,-5.950845e-02,-2.599606e-02,8.319720e-03,-1.365920e-02,-3.325456e-02,-4.574921e-02,3.177380e-02,-5.096604e-02,-4.314130e-02,-3.141887e-02,-4.732519e-02,2.315413e-02,-5.909326e-03,-4.481348e-02,-3.496681e-02,1.164693e-02,1.930233e-02,-1.971319e-02,1.215008e-02,-5.624398e-03,-1.913277e-02,3.169686e-02,1.922367e-02,1.148034e-02,-1.157094e-02,-4.326063e-02,-4.094508e-02,-3.960027e-02,7.054209e-03,-2.693577e-02,-1.923215e-02,-2.866463e-02,1.814574e-02,7.407875e-03,-5.573149e-03,-3.008469e-02,4.067311e-02,-1.446838e-02,-3.855599e-02,7.407875e-03,-1.645849e-02,1.910191e-02,-5.617454e-02,7.159523e-03,-5.331636e-02,-4.302211e-02,-2.052196e-02,2.473041e-02,NaN,NaN,-1.602908e-02,-1.602908e-02,NaN,-2.693126e-02,2.231065e-02,2.874511e-02,-1.887774e-02,2.874511e-02,-1.887774e-02,-6.964616e-02,NaN,-6.672324e-02,-6.751540e-02,NaN,NaN,NaN,NaN,-1.617821e-02,1.403572e-02,NaN,-4.435376e-03,NaN,-4.449769e-03,NaN,-2.643715e-02,-3.003536e-03,-2.477989e-02,NaN,2.619155e-02,5.207066e-03,-1.518957e-02,-4.145817e-02,-1.376796e-02,-4.357384e-02,1.864922e-02,-3.170211e-02,1.484370e-02,NaN,3.429740e-02,1.484370e-02,-1.851950e-02,1.317104e-02,NaN,-4.188734e-14,3.388881e-14
PID,2.473041e-02,1.000000e+00,-1.041557e-01,-2.307220e-01,-1.733455e-01,-5.137517e-03,-9.733961e-02,3.975128e-02,-2.521859e-01,1.181672e-01,-3.381518e-01,-1.306511e-01,-2.307681e-01,-1.169371e-01,-4.594102e-03,-7.125072e-02,-1.935949e-01,-1.559601e-01,1.254433e-02,6.010266e-02,-4.622794e-02,1.530048e-03,-1.569921e-01,-1.706261e-01,1.722285e-03,4.848923e-02,-7.629927e-02,-1.085388e-01,-2.592638e-01,-2.242084e-01,-1.996926e-01,-4.185396e-02,-6.044447e-02,1.723710e-01,-2.063369e-02,-3.017522e-02,-6.237739e-03,-2.164846e-02,-6.479384e-02,1.445505e-03,-2.356591e-01,-2.003570e-01,-6.237739e-03,-1.685818e-01,1.147612e-03,-1.052152e-01,2.616954e-02,-9.636566e-02,-1.286569e-01,-1.921321e-01,1.000000e+00,NaN,NaN,-3.798564e-01,-3.798564e-01,NaN,1.265750e-02,5.093953e-01,8.954693e-03,-1.453070e-02,8.954693e-03,-1.453070e-02,-2.209204e-01,NaN,-2.194978e-01,-2.200548e-01,NaN,NaN,NaN,NaN,3.973518e-02,1.817990e-02,NaN,-3.999051e-02,NaN,-2.317678e-02,NaN,3.536129e-02,-3.992417e-03,3.583599e-02,NaN,-4.005048e-02,-1.841044e-01,-9.754585e-02,-1.895296e-01,-1.559222e-01,-2.189193e-01,5.739454e-02,-1.243799e-02,-1.696028e-01,NaN,-1.949638e-03,-1.696028e-01,-8.052447e-01,-2.005123e-01,NaN,-6.357451e-16,-2.145736e-16
GrLivArea,-1.517788e-02,-1.041557e-01,1.000000e+00,7.210624e-01,1.687778e-02,6.241574e-02,3.508861e-01,2.641483e-01,5.636358e-01,-1.103698e-01,2.312376e-01,3.072151e-01,3.864757e-01,1.675433e-01,-1.584913e-02,2.468327e-01,4.144304e-01,5.430575e-01,6.618841e-01,8.555649e-02,3.970224e-02,-5.285102e-02,6.439025e-01,4.419142e-01,5.366928e-01,1.302555e-01,8.087598e-01,4.610701e-01,2.675509e-01,4.968049e-01,4.775104e-01,2.437328e-01,3.259185e-01,2.261652e-03,9.215582e-03,9.943540e-02,5.796283e-02,-3.095299e-03,3.830608e-02,-3.629839e-03,2.668675e-01,4.763890e-01,5.796283e-02,4.130938e-01,-5.888990e-02,2.130573e-01,2.656733e-02,2.16

In [ ]:
housing.corr()['SalePrice'].sort_values()

In [ ]:
housing.shape

In [ ]:
housing.info()

In [ ]:
pd.set_option('display.max_rows',None)
house_df.shape

In [ ]:
real_estate.isnull().sum()

In [ ]:
house_df.head()

In [ ]:
pd.set_option('display.max_rows',None)

house_df.isnull().sum()

## Features that drop because of too many missing values
- Alley
- PoolArea
- PoolQC
- Fence
- MiscFeature
- MiscVal
- YrSold_YYYY
- MoSold_MM
- SaleCond
- PA-PreD
- PA-PostD
- PA-UnTyp
- PA-UntNo
- X1TPr_D
- X1TSc_D
- X2TPr_D
- X2TSc_D
- X1TPr_S
- X1TSc_S
- X2TPr_S
- X2TSc_S

## Features that drop because of multicollinearity
- ISU_lat_long
- address_lat_long
- FullBath 
- HalfBath
- MA_Ownr1
- MA_Ownr2
- MA_Line1
- MA_Line2
- MA_City
- MA_State
- location
- point


## Features that drop because of p value
- MasVnrArea2
- BsmtQual
- BsmtCond
- BsmtExposure
- Electrical

In [ ]:
house_df[house_df['PA-UntNo'].isnull()==False]

In [ ]:
house_df['PA-Nmbr']

In [ ]:
house_df['PA-Strt']

In [ ]:
house_df['MA_Line1']

In [ ]:
house_df['address']

In [ ]:
len(house_df['Distance'])

In [ ]:
housing

In [ ]:
real_estate.columns

In [ ]:
pd.value_counts(housing_df['MapRefNo'] == housing_df['GeoRefNo']) # MapRefNo same as GeoRefNo, so drop these two columns

In [ ]:
housing_df.shape

## SalePrice difference??

In [ ]:
housing_df['SalePrice_y']

In [ ]:
housing_df['SalePrice_x']

In [ ]:
housing_df[housing_df['PID'].duplicated()]

In [ ]:
housing_df.head()

In [ ]:
housing_df[housing_df['PID']==907135040][['SalePrice_x','SalePrice_y']]

In [ ]:
print(len(set(housing['PID']))) # 2579 unique observations
print(len(set(housing_df['PID']))) # 2558 unique observations
# housing_df.drop(['Map'])

In [ ]:
housing_df[housing_df['PID']==909276070]

In [ ]:
len(set(housing_df['PID']))

In [ ]:
housing[housing['PID']==909276070]

In [ ]:
correlation = housing.corr()['SalePrice']
# corr_matrix['SalePrice']
correlation.sort_values()

Danny

In [ ]:
housing3 = pd.concat([housing[housing.columns[53:]],housing['SalePrice']],axis=1)
housing3.columns

In [ ]:
housing_features = housing[housing.columns[53:]]
housing_features = housing_features.rename(columns= {"3SsnPorch":"X3SsnPorch"})
housing_featuresClass = housing_features[['KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual',\
                                         'GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']]
housing_featuresQuan = housing_features[['KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea',\
                                        'WoodDeckSF','OpenPorchSF','EnclosedPorch','X3SsnPorch','ScreenPorch','PoolArea',\
                                         'MiscVal','MoSold','YrSold']]

In [ ]:
housing_features.columns

In [ ]:
for feature1 in housing_featuresClass.columns:
    print(housing3[feature1].value_counts(), housing_featuresClass[feature1].isnull().value_counts())
    print('*'*30)

In [ ]:
house_df['Fence']

In [ ]:
for feature2 in housing_featuresQuan.columns:
    print(housing_featuresQuan[feature2].isnull().value_counts())
    print('*'*30)

In [ ]:
housing3['Fireplaces'].value_counts()

In [ ]:
housing3['FireplaceQu'] = housing3['FireplaceQu'].fillna('None')

In [ ]:
housing3[housing3['GarageYrBlt']=='nan']

In [ ]:
# Impute None to Nan value in columns
impute_feature = housing3[['FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']]
for feature in impute_feature.columns:
    housing3[feature] = housing3[feature].fillna('None')

In [ ]:
# housing3['GarageType'].value_counts()

In [ ]:
housing3[housing3['GarageType'].isnull() == True].T

In [ ]:
# Drop the Nan from GarageCars column (1 observation)
housing3.drop(housing3[housing3['GarageCars'].isnull() == True].index,inplace=True)
housing3.columns

In [ ]:
housing3.isnull().sum()

In [ ]:
housing3.head()

In [ ]:
X.isnull().sum().sum()

#### PoolArea, PoolQC, Fence, MiscFeature, MiscVal will be drop due to too many missing value

In [ ]:
housing_features2 = housing3.drop(['PoolArea','PoolQC','Fence','MiscFeature','MiscVal','SalePrice'],axis=1)
len(housing_features2.columns)

In [ ]:
X = pd.get_dummies(housing_features2, drop_first=False, dummy_na=True)
price = housing3['SalePrice']
price.shape

In [ ]:
pd.to_numeric(housing_features2['GarageYrBlt'])

In [ ]:
X.shape

In [ ]:
price.shape

In [ ]:
import statsmodels.api as sm
x_feature = sm.add_constant(X)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(price, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

In [ ]:
X.reset_index()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,price,test_size=0.2,random_state = 60)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
lin_reg = LinearRegression().fit(X_train,y_train)
lin_reg.score(X_train,y_train)

In [ ]:
y_test

In [ ]:
from sklearn.metrics import r2_score
lin_reg_pred = lin_reg.predict(X_test)
# sum((y_test - lin_reg_pred)**2)/sum((y_test - np.mean(y_test))**2)
# mean_squared_error(X_test,y_test)
# lin_reg_pred
# len(y_test)
r2_score(y_test,lin_reg_pred)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# housing_prediction = lin_reg.predict(X_test)
# mean_squared_error(y_train,y_test)
lin_reg2 = LinearRegression().fit(X_train,y_train)
lin_reg.r2_score

In [ ]:
# lin_reg = LinearRegression().fit(X_test,y_test)
lin_reg.score(X_test,y_test)

In [ ]:
lin_reg1 = LinearRegression()
lin_reg1.score(X_train,y_train)

In [ ]:

lin_reg.score(y_test,lin_reg_pred)

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=10).fit(X_train,y_train)
tree_reg.score(X_train,y_train)

In [ ]:
# tree_reg = DecisionTreeRegressor(max_depth=10).fit(X_test,y_test)
tree_reg.score(X_test,y_test)

In [ ]:
print(tree_reg.feature_importances_)

In [ ]:
forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X_train,y_train)
forest_reg.score(X_train,y_train)

In [ ]:
# forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X_test,y_test)
forest_reg.score(X_test,y_test)

In [ ]:
from geopy.distance import geodesic
test1 = 


geodesic